In [1]:
import records
import json

In [2]:
db = records.Database('sqlite:///:memory:')

In [3]:
db.query("drop table if exists pc_choice")
db.query("drop table if exists playercharacter")
db.query("drop table if exists subselection;")
db.query("drop table if exists selection;")
db.query("drop table if exists choice;")
db.query("drop table if exists trait;")
db.query("drop table if exists source;")

db.query("""create table source (
         id integer primary key autoincrement,
         fulltitle text not null,
         abbreviation text not null,
         collection text not null
         )""")
db.query("""create table selection (
         id integer primary key autoincrement,
         title text not null unique
        )""")
db.query("""create table choice (
         id integer primary key autoincrement,
         title text not null,
         source_id int,
         selection_id int not null,
         foreign key(source_id) references source(id),
         foreign key(selection_id) references selection(id),
         unique( title, selection_id)
         )""")
db.query("""create table trait (
         id integer primary key autoincrement,
         title text not null,
         description text not null,
         level_acquired int not null,
         choice_id int not null,
         foreign key(choice_id) references choice(id)
         )""")
db.query("""create table subselection (
         id integer primary key autoincrement,
         title text not null,
         choice_id int not null,
         selection_id int not null,
         foreign key(choice_id) references choice(id),
         foreign key(selection_id) references selection(id),
         unique (choice_id, selection_id)
         )""")
db.query("""create table playercharacter(
         id integer primary key autoincrement,
         pc_name text not null,
         strength integer not null,
         dexterity integer not null,
         constitution integer not null,
         intelligence integer not null,
         wisdom integer not null,
         charisma integer not null,
         level integer not null,
         prof_acrobatics int,
         prof_animal_handling int,
         prof_arcana int,
         prof_athletics int,
         prof_deception int,
         prof_history int,
         prof_insight int,
         prof_intimidation int,
         prof_investigation int,
         prof_medicine int,
         prof_nature int,
         prof_perception int,
         prof_performance int,
         prof_persuasion int,
         prof_religion int,
         prof_sleight_of_hand int,
         prof_stealth int,
         prof_survival int
         )""")
db.query("""create table pc_choice (
         id integer primary key autoincrement,
         pc_id integer not null,
         choice_id integer not null,
         foreign key(pc_id) references playercharacter(id),
         foreign key(choice_id) references choice(id),
         unique (pc_id, choice_id)
         )""")

<RecordCollection size=0 pending=True>

In [4]:
with open('source.json') as f:
    sources=json.load(f)

In [5]:
for _,d in sources.items():
    db.query('insert into source (fulltitle, abbreviation, collection) values (:name,:abbreviation,:collection)',
             name=d['name'],abbreviation=d['abbreviation'],collection=d['group']) 

In [6]:
db.query('insert into selection (title) values ("Race")')
with open('race.json') as f:
    races=json.load(f)

In [7]:
selection_id=db.query('select id from selection where title="Race";')[0]['id']
for race in races['race']:
    source_id=db.query('select id from source where fulltitle like :source',
                       source="%"+race['source'])[0]['id']
    db.query('insert into choice (title, source_id, selection_id) values (:name,:source_id,:selection_id)',
             name=race['name'],source_id=source_id,selection_id=selection_id)
    choice_id=db.query('select id from choice where title=:name',name=race['name'])[0]['id']
    db.query('insert into trait (title,description,level_acquired,choice_id) values ("Speed",:speed, 0, :choice_id)',
             speed=race['speed'],choice_id=choice_id)
    db.query('insert into trait (title,description,level_acquired,choice_id) values ("Size",:size, 0, :choice_id)',
             size=race['size'],choice_id=choice_id)
    db.query('insert into trait (title, description, level_acquired, choice_id) values ("Ability",:ability, 0, :choice_id)',
             ability=race['ability'], choice_id=choice_id)
    for trait in race['trait']:
        db.query('insert into trait (title, description, level_acquired, choice_id) values (:name, :text, 0, :choice_id)',
                name=trait['name'], text='\n'.join(t for t in trait['text'] if t), choice_id=choice_id)

In [8]:
db.query('insert into selection (title) values ("Background")')
with open('background.json') as f:
    backgrounds=json.load(f)

In [9]:
selection_id = db.query('select id from selection where title="Background";')[0]['id']
for background in backgrounds['background']:
    source_id=db.query('select id from source where fulltitle like :source',
                      source="%"+race['source'])[0]['id']
    db.query('insert into choice (title, source_id, selection_id) values (:name, :source_id, :selection_id)',
            name=background['name'],source_id=source_id,selection_id=selection_id)
    choice=db.query('select * from choice where title=:name', name=background['name'])
    for trait in background['trait']:
        db.query('insert into trait (title, description, level_acquired, choice_id) values'\
                 '(:name, :text, 0, :choice_id)',
                name=trait['name'], text='\n'.join(t for t in trait['text'] if t), choice_id=choice[0]['id'])

In [10]:
row=db.query('select * from choice')
for r in row:
    print(r)

<Record {"id": 1, "title": "Aarakocra", "source_id": 4, "selection_id": 1}>
<Record {"id": 2, "title": "Genasi (Air)", "source_id": 4, "selection_id": 1}>
<Record {"id": 3, "title": "Elf (Drow)", "source_id": 1, "selection_id": 1}>
<Record {"id": 4, "title": "Gnome (Deep)", "source_id": 4, "selection_id": 1}>
<Record {"id": 5, "title": "Dragonborn", "source_id": 1, "selection_id": 1}>
<Record {"id": 6, "title": "Genasi (Earth)", "source_id": 4, "selection_id": 1}>
<Record {"id": 7, "title": "Elf (Eladrin)", "source_id": 2, "selection_id": 1}>
<Record {"id": 8, "title": "Genasi (Fire)", "source_id": 4, "selection_id": 1}>
<Record {"id": 9, "title": "Gnome (Forest)", "source_id": 1, "selection_id": 1}>
<Record {"id": 10, "title": "Goliath", "source_id": 4, "selection_id": 1}>
<Record {"id": 11, "title": "Half-Elf", "source_id": 1, "selection_id": 1}>
<Record {"id": 12, "title": "Half-Orc", "source_id": 1, "selection_id": 1}>
<Record {"id": 13, "title": "Elf (High)", "source_id": 1, "sele

/home/jriveros/Documents/projects/python/venvs/charbuild/lib64/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: generator 'RecordCollection.__iter__' raised StopIteration
  


In [11]:
db.query('insert into selection (title) values ("Class")')
with open('class.json') as f:
    classes=json.load(f)

In [12]:
for cclass in classes['class']:
    selection_id=db.query('select id from selection where title="Class"')[0]['id']
    source_id=db.query('select id from source where fulltitle like :source', source="%"+cclass['source'])[0]['id']
    db.query('insert into choice (title, source_id, selection_id) values (:name, :source, :selection)',name=cclass['name'],source=source_id,selection=selection_id)
    choice_id=db.query('select id from choice where title=:name', name=cclass['name'])[0]['id']
    for level in cclass['autolevel']:
        if 'feature' in level:
            for feature in level['feature']:
                if 'subclass' not in feature:
                    db.query('insert into trait (title, description, level_acquired, choice_id) values (:name, :text, :level, :choice_id)',name=feature['name'], text='\n'.join(t for t in feature['text'] if t),level=level['_level'],choice_id=choice_id)
                elif 'issubclass' in feature:
                    subselection=[r for r in db.query('select * from selection where title=:parent',parent=feature['parent'])]
                    if not subselection:
                        db.query('insert into selection (title) values (:parent)', parent=feature['parent'])
                        subselection=db.query('select * from selection where title=:parent',parent=feature['parent'])
                    db.query('insert into choice (title,selection_id) values (:feature,:subselection)', feature=feature['name'], subselection=subselection[0]['id'])
                    db.query('insert into trait (title, description, level_acquired, choice_id) values (:name, :text, :level, :subselection)', name=feature['name'],text='\n'.join(t for t in feature['text'] if t), level=level['_level'],subselection=subselection[0]['id'])
                else:
                    try:
                        subchoice = [r for r in db.query('select * from choice where title=:parent', parent=feature['subclass'])]
                        db.query('insert into trait (title, description, level_acquired, choice_id) values (:name, :text, :level, :subchoice)', name=feature['name'],text='\n'.join(t for t in feature['text'] if t), level=level['_level'], subchoice=subchoice[0]['id'])
                    except:
                        print('##',feature['name'])

## City Domain: Bonus Proficiencies


/home/jriveros/Documents/projects/python/venvs/charbuild/lib64/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: generator 'RecordCollection.__iter__' raised StopIteration
  if sys.path[0] == '':
/home/jriveros/Documents/projects/python/venvs/charbuild/lib64/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: generator 'RecordCollection.__iter__' raised StopIteration


## Arcane Archer: Ever-Ready Shot
## Arcane Archer: Extra Arcane Shot


for cclass in classes['class']:
    selection_id = db.query('select id from selection where title="Class";')[0]['id']
    source_id=db.query('select id from source where fulltitle like :source',
                      source="%"+cclass['source'])[0]['id']
    
    db.query('insert into choice (title, source_id, selection_id) values (:name, :source_id, :selection_id)',
            name=cclass['name'],source_id=source_id,selection_id=selection_id)
    choice_id=db.query('select id from choice where title=:name', name=cclass['name'])[0]['id']
    for level in cclass['autolevel']:
        
        if 'feature' in level:
            for feature in level['feature']:
                
                if 'subclass' not in feature:
                    db.query('insert into trait (title, description, level_acquired, choice_id) values'\
                            '(:name, :text, :level, :choice_id)',
                            name=feature['name'], 
                            text='\n'.join(t for t in feature['text'] if t),
                            level=level['_level'],
                            choice_id=choice_id)
                else:
                    print('sub class feature:',feature['parent'],'(',feature['name'],')')
                    subselect_id= [row for row in db.query('select * from selection where title=:title',
                                  title=feature['parent'])]
                    if subselect_id:
                        print(feature['parent'],'has selection.',feature['name'],'will be added.')
                        # If selection already exists
                        subchoice_id = [
                            row for row in db.query('select * from choice where '\
                                                    'title=:subclass and selection_id=:subselect_id',
                                                    subclass=feature['parent'], subselect_id=subselect_id[0]['id'])]
                        if subchoice_id:
                            print('##Adding',feature['name'])
                            db.query('insert into trait (title, description, level_acquired, choice_id) values'\
                                     '(:title, :description, :level_acquired, :choice_id)',
                                    title=feature['name'],
                                    description='\n'.join(t for t in feature['text'] if t),
                                    level_acquired=level['_level'],
                                    choice_id=subchoice_id[0]['id'])
                        else:
                            subselect_id=db.query('select * from selection where title=:title',title=feature['parent'])    
                            db.query('insert into choice (title, selection_id) values (:title, :selection)',
                                    title=feature['subclass'],selection=subselection_id[0]['id'])

                    else:
                        print("##Creating Selection for",feature['name'])
                        db.query('insert into selection (title) values (:title)', title=feature['parent'])
                        subselect_id=db.query('select * from selection where title=:title',title=feature['parent'])
                        db.query('insert into choice (title,selection_id) values (:title,:selection)',
                                title=feature['subclass'],selection=subselect_id[0]['id'])
                        subchoice_id=db.query('select * from choice where title=:title and selection_id=:subselect',
                                             title=feature['subclass'],
                                             subselect=subselect_id[0]['id'])
                        db.query('insert into subselection (title, choice_id, selection_id) values'\
                                 '(:title, :choice_id, :selection_id)',
                                title=feature['parent'],
                                choice_id=subchoice_id[0]['id'],
                                selection_id=subselect_id[0]['id'])
                        db.query('insert into trait (title, description, level_acquired, choice_id) values'\
                                     '(:title, :description, :level_acquired, :choice_id)',
                                    title=feature['name'],
                                    description='\n'.join(t for t in feature['text'] if t),
                                    level_acquired=level['_level'],
                                    choice_id=subchoice_id[0]['id'])
                

In [44]:
class PC:
    def __init__(self):
        self.choices = dict()
        self.traits = dict()
        self.passedselections = list()
        self.characterlevel = lambda : sum(self.classlevels.values())
        self.classlevels = {
             'Artificer (UA)': 0,
             'Barbarian': 0,
             'Bard': 0,
             'Cleric': 0,
             'Druid': 0,
             'Fighter': 0,
             'Monk': 0,
             'Mystic (UA)': 0,
             'Paladin': 0,
             'Ranger': 0,
             'Ranger (Revised)': 0,
             'Rogue': 0,
             'Sorcerer': 0,
             'Warlock': 0,
             'Wizard': 0
        }

In [45]:
def multiclassFilter(choices, pc):
    remaining = [cl['title'] for cl in choices]
    for cl in choices:
        try:
            if pc.strength < 13:
                remaining.remove('Barbarian')
                remaining.remove('Paladin')
            if pc.dexterity < 13:
                remaining.remove('Monk')
                remaining.remove('Ranger')
                remaining.remove('Ranger (Revised)')
                remaining.remove('Rogue')
            if pc.wisdom < 13:
                remaining.remove('Cleric')
                remaining.remove('Druid')
                remaining.remove('Monk')
                remaining.remove('Ranger')
                remaining.remove('Ranger (Revised)')
            if pc.charisma < 13:
                remaining.remove('Bard')
                remaining.remove('Paladin')
                remaining.remove('Sorcerer')
                remaining.remove('Warlock')
            if pc.strength < 13 and pc.dexterity < 13:
                remaining.remove('Fighter')
            if pc.intelligence < 13:
                remaining.remove('Wizard')
                remaining.remove('Mystic (UA)')
                remaining.remove('Artificer (UA)')
        except:
            pass
    return [cl for cl in choices if cl['title'] in remaining]

In [51]:
level=int(input('Character Level: '))
q=list('Class' for _ in range(level))
q+=['Background','Race',]
pc=PC()
pc.strength = 13
pc.dexterity = 11
pc.constitution = 12
pc.intelligence = 14
pc.wisdom = 15
pc.charisma = 15

while q:
    INPUT=q.pop()
    try:
        Choices=list(dict(r) for r in db.query('select * from choice as c join selection as s on c.selection_id=s.id where s.title=:title',
                       title=INPUT))
    except:
        pass
    if INPUT == 'Class' and pc.characterlevel != 0:
        Choices=multiclassFilter(Choices, pc)
    for r in Choices:
        print(r['id'],'-',r['title'])
    value = None
    while not value:
        value=int(input(f"{INPUT}: "))
        if value not in [r['id'] for r in Choices]:
            value=None
            
    choiceTitle=[r['title'] for r in Choices if r['id'] == value][0]
    pc.choices[value]=choiceTitle
    if INPUT == 'Class':
        pc.classlevels[choiceTitle]+=1
        rellevel = pc.classlevels[choiceTitle]
    else:
        rellevel = pc.characterlevel()
    traits = db.query('select * from trait where choice_id=:choice_id and level_acquired <:level',choice_id=value, level=rellevel+1)
    pc.traits[choiceTitle]=dict()
    pc.passedselections.append(INPUT)
    for t in traits:
        pc.traits[choiceTitle][t['title']]=t['description']
        tmpQ=[r for r in db.query('select * from selection where title=:title',title=t['title'])]
        if tmpQ:
            for r in tmpQ:
                if r['id'] not in pc.choices and r['title'] not in pc.passedselections:
                    q.append(r['title'])

Character Level: 3


/home/jriveros/Documents/projects/python/venvs/charbuild/lib64/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: generator 'RecordCollection.__iter__' raised StopIteration
  from ipykernel import kernelapp as app


1 - Aarakocra
2 - Genasi (Air)
3 - Elf (Drow)
4 - Gnome (Deep)
5 - Dragonborn
6 - Genasi (Earth)
7 - Elf (Eladrin)
8 - Genasi (Fire)
9 - Gnome (Forest)
10 - Goliath
11 - Half-Elf
12 - Half-Orc
13 - Elf (High)
14 - Dwarf (Hill)
15 - Human
16 - Human (Variant)
17 - Halfling (Lightfoot)
18 - Dwarf (Mountain)
19 - Shifter (Razorclaw)
20 - Gnome (Rock)
21 - Halfling (Stout)
22 - Tiefling (Infernal)
23 - Genasi (Water)
24 - Shifter (Wildhunt)
25 - Elf (Wood)
26 - Dwarf (Duergar)
27 - Half-Elf (Wood Elf Descent)
28 - Half-Elf (Moon Elf or Sun Elf Descent)
29 - Half-Elf (Drow Descent)
30 - Half-Elf (Aquatic Elf Descent)
31 - Halfling (Ghostwise)
32 - Bugbear
33 - Aasimar (Fallen)
34 - Aasimar (Protector)
35 - Aasimar (Scourge)
36 - Firbolg
37 - Goblin
38 - Hobgoblin
39 - Kenku
40 - Kobold
41 - Lizardfolk
42 - Orc
43 - Tabaxi
44 - Triton
45 - Yuan-ti Pureblood
46 - Changeling
47 - Shifter (Beasthide)
48 - Shifter (Cliffwalk)
49 - Shifter (Longstride)
50 - Shifter (Longtooth)
51 - Warforged
52 -

/home/jriveros/Documents/projects/python/venvs/charbuild/lib64/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: generator 'RecordCollection.__iter__' raised StopIteration
/home/jriveros/Documents/projects/python/venvs/charbuild/lib64/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: generator 'RecordCollection.__iter__' raised StopIteration


81 - Acolyte
82 - Charlatan
83 - City Watch
84 - Clan Crafter
85 - Cloistered Scholar
86 - Courtier
87 - Criminal
88 - Entertainer
89 - Faction Agent
90 - Far Traveler
91 - Folk Hero
92 - Guild Artisan
93 - Hermit
94 - Inheritor
95 - Investigator
96 - Knight of the Order
97 - Mercenary Veteran
98 - Noble
99 - Outlander
100 - Sage
101 - Sailor
102 - Soldier
103 - Urban Bounty Hunter
104 - Urchin
105 - Uthgardt Tribe Member
106 - Waterdhavian Noble
107 - Variant Criminal (Spy)
108 - Variant Entertainer (Gladiator)
109 - Variant Guild Artisan (Guild Merchant)
110 - Variant Noble (Knight)
111 - Variant Sailor (Pirate)
112 - Inquisitor
113 - Haunted One
114 - Initiate
115 - Vizier
Background: 97
116 - Barbarian
124 - Bard
132 - Cleric
151 - Druid
158 - Fighter
182 - Paladin
214 - Sorcerer
225 - Warlock
236 - Wizard
251 - Artificer (UA)
254 - Mystic (UA)
Class: 124
116 - Barbarian
124 - Bard
132 - Cleric
151 - Druid
158 - Fighter
182 - Paladin
214 - Sorcerer
225 - Warlock
236 - Wizard
251 - 

for k,v in pc.traits.items():
    print(k)
    print(v)
    print('\n\n')

In [15]:
tmpQ

[]

In [52]:
for k,v in pc.traits.items():
    print(k)
    for kk,vv in v.items():
        print(kk)
        print('\t',vv,'\n')
    print('\n\n')

Human (Variant)
Speed
	 30 

Size
	 M 

Ability
	 Choose two 1 

Ability Score Increase
	 Two different ability scores of your choice increase by 1. 

Skills
	 You gain proficiency in one skill of your choice. 

Feat
	 You gain one feat of your choice. 

Languages
	 You can speak, read, and write Common and one extra language of your choice. Humans typically learn the languages of other peoples they deal with, including obscure dialects. They are fond of sprinkling their speech with words borrowed from other tongues: Orc curses, Elvish musical expressions, Dwarvish military phrases, and so on. 




Mercenary Veteran
Skill Proficiencies
	 Athletics, Persuasion 

Tool Proficiencies
	 One type of gaming set, vehicles (land) 

Equipment
	 A uniform of your company (traveler's clothes in quality), an insignia of your rank, a gaming set of your choice, and a pouch containing the remainder of your last wages (10 gp). 

Feature: Mercenary Life
	 You know the mercenary life as only someone who 

In [22]:
classes=list(dict(r) for r in db.query('select * from choice as c join selection as s on c.selection_id=s.id where s.title=:title',
                       title='Class'))

/home/jriveros/Documents/projects/python/venvs/charbuild/lib64/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: generator 'RecordCollection.__iter__' raised StopIteration
  """Entry point for launching an IPython kernel.


SyntaxError: unexpected EOF while parsing (<ipython-input-21-89e470a66a34>, line 2)

In [38]:
dict(zip([i['title'] for i in classes],(0 for _ in range(15))))


{'Artificer (UA)': 0,
 'Barbarian': 0,
 'Bard': 0,
 'Cleric': 0,
 'Druid': 0,
 'Fighter': 0,
 'Monk': 0,
 'Mystic (UA)': 0,
 'Paladin': 0,
 'Ranger': 0,
 'Ranger (Revised)': 0,
 'Rogue': 0,
 'Sorcerer': 0,
 'Warlock': 0,
 'Wizard': 0}